In [1]:
# Loading the required libraries

import os
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'BCS_CNN_AUG_final.ipynb',
 'BCS_confusion_matrix_final.ipynb',
 'BCS_Data_Augmentation_final.ipynb',
 'BCS_training_resnet101_final.ipynb',
 'BCS_training_vgg16_final.ipynb',
 'Feature extraction.ipynb',
 'k-means.ipynb',
 'npy_test_preprocess.ipynb',
 'Train-Validation Split_final.ipynb',
 'vizualization_final.ipynb']

In [2]:
# Image Augmentation  
img_width=256; img_height=256
batch_size=16

TRAINING_DIR = '/insofe/Projects/Some_Project/Tensorflow/BCS challenge/Chandra_BCS_Challange/data/train/'

train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True )

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))



VALIDATION_DIR = '/insofe/Projects/Some_Project/Tensorflow/BCS challenge/Chandra_BCS_Challange/data/valid/'

validation_datagen = ImageDataGenerator(rescale = 1/255.0)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              target_size=(img_height, img_width)
                                                             )

Found 2466 images belonging to 4 classes.
Found 719 images belonging to 4 classes.


In [6]:
# creating callbacks,early stopping & saving the file
callbacks = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
best_model_file = '/insofe/Projects/Some_Project/Tensorflow/BCS challenge/Chandra_BCS_Challange/new_models/CNN.h5'
best_model = ModelCheckpoint(best_model_file, monitor='val_acc', verbose = 1, save_best_only = True)

In [7]:
# building the model architecture
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(img_width,img_height,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics =['accuracy'])


In [8]:
# fitting the model
history = model.fit_generator(train_generator,
                              epochs=30,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [best_model]
                              )

C:\Users\saira\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
155/155 [==============================] - 325s 2s/step - loss: 1.2048 - accuracy: 0.6985 - val_loss: 0.9400 - val_accuracy: 0.6996
Epoch 2/30
155/155 [==============================] - 304s 2s/step - loss: 0.8135 - accuracy: 0.7272 - val_loss: 0.9189 - val_accuracy: 0.6996
Epoch 3/30
155/155 [==============================] - 302s 2s/step - loss: 0.7755 - accuracy: 0.7409 - val_loss: 0.8980 - val_accuracy: 0.6996
Epoch 4/30
155/155 [==============================] - 301s 2s/step - loss: 0.7621 - accuracy: 0.7328 - val_loss: 0.9156 - val_accuracy: 0.6996
Epoch 5/30
155/155 [==============================] - 300s 2s/step - loss: 0.7852 - accuracy: 0.7322 - val_loss: 0.8853 - val_accuracy: 0.6996
Epoch 6/30
155/155 [==============================] - 306s 2s/step - loss: 0.7880 - accuracy: 0.7305 - val_loss: 0.8960 - val_accuracy: 0.6996
Epoch 7/30
155/155 [==============================] - 301s 2s/step - loss: 0.7436 - accuracy: 0.7457 - val_loss: 0.9492 - val_accuracy: 0.6996

##### With the Base model the validation not changing....but the validation loss is fluktuating uo and down.